In [37]:
import pandas as pd
import seaborn as sns


df = pd.read_csv("T_games_dataset.csv")
df.head(1)

,id,client_id,order_day,category_id,category_name,good_id,good_name,good_price,good_cnt,gender_cd,age,education_level,city_nm,monthly_income_amt,steam_popularity_score
0,1500571355,5-25R6ZT3L9,2023-05-22,29688,Скидки,890257,The Night of the Rabbit,299.0,1.0,M,28,UGR,Москва,73552,0.0


In [38]:
df = df[df.monthly_income_amt > 0]

In [39]:
from functools import lru_cache


@lru_cache(None)
def set_region(name: str) -> str:
    if name in ["Москва", "Санкт-Петербург"]:
        return "Столица"

    return "Регионы"


df["region"] = df.city_nm.apply(set_region)
df["age_group"] = df.age.apply(lambda x: f"{10*(x // 10)}-{10*(x // 10 + 1)-1}") # Приводим к виду 10-19

def set_income_group(income: int) -> str:
    if income < 30_000:
        return "<30 000"

    if income > 300_000:
        return ">300 000"

    income -= 30_000

    return f"{20_000*(income // 20_000) + 30_000}-{20_000*(income // 20_000 + 1) + 30_000}"

df["income_group"] = df.monthly_income_amt.apply(set_income_group)
df["gender_cd"] = df.gender_cd.map({"F": "Жен", "M": "Муж"})
df["education_level"] = df.education_level.map({
    "SCH": "Начальное, среднее образование",
    "UGR": "Неполное высшее образование",
    "GRD": "Высшее образование",
    "PGR": "Два высших образования",
    "ACD": "Ученая степень",
})

In [40]:
from datetime import datetime
from dateutil import relativedelta

start_date = datetime.strptime(df.order_day.min(), "%Y-%m-%d")
end_date = datetime.strptime(df.order_day.max(), "%Y-%m-%d")

delta = relativedelta.relativedelta(end_date, start_date)
# Так как осталось 22 дня, то защитаю это за еще один месяц,
# либо можно убрать эти заказы из выборки
total_months = delta.years * 12 + delta.months

delta

relativedelta(years=+1, months=+8, days=+22)

In [41]:
r = df.groupby([
    "region", "age_group",
    "income_group", "gender_cd",
    "education_level"
]).good_price.sum().reset_index()

r.good_price /= total_months

# r.columns = [
#     "Мето проживания",
#     "Возрастаня группа",
#     "Доход",
#     "Пол",
#     "Образование",
#     "Сумма, приносимая бизнесу в месяц"
# ]

expenses = r.copy()
r.sort_values("good_price", ascending=False).head(10)

,region,age_group,income_group,gender_cd,education_level,good_price
889,Столица,20-29,50000-70000,Муж,"Начальное, среднее образование",46031.1165
887,Столица,20-29,50000-70000,Муж,Высшее образование,38509.4000
916,Столица,20-29,<30 000,Муж,"Начальное, среднее образование",31194.7160
880,Столица,20-29,30000-50000,Муж,"Начальное, среднее образование",30945.5430
255,Регионы,20-29,50000-70000,Муж,"Начальное, среднее образование",30705.3405
914,Столица,20-29,<30 000,Муж,Высшее образование,29155.6500
896,Столица,20-29,70000-90000,Муж,Высшее образование,28378.9755
890,Столица,20-29,50000-70000,Муж,Неполное высшее образование,27758.6285
1027,Столица,30-39,50000-70000,Муж,Высшее образование,27618.2745
932,Столица,30-39,110000-130000,Муж,Высшее образование,26256.4735


In [42]:
r = df.drop(df[df.client_id.duplicated()].index)
r = r.groupby([
    "region", "age_group",
    "income_group", "gender_cd",
    "education_level"
]).education_level.value_counts()

r = r.reset_index()

total_people = r["count"].sum()
r["proportion"] = r["count"] / total_people
r["proportion"] = r["proportion"].round(5)

expenses = expenses.merge(r, on=["region", "age_group", "income_group", "gender_cd", "education_level"])
expenses["sum_per_person"] = expenses["good_price"] / expenses["count"]
expenses.to_csv("expenses_by_groups.csv")

In [43]:
r = expenses.sort_values("good_price", ascending=False).head(15)
r.sort_values("sum_per_person", ascending=False)

,region,age_group,income_group,gender_cd,education_level,good_price,count,proportion,sum_per_person
848,Столица,30-39,110000-130000,Муж,Высшее образование,26256.4735,311,0.00839,84.425960
934,Столица,30-39,50000-70000,Муж,Высшее образование,27618.2745,358,0.00965,77.146018
815,Столица,20-29,70000-90000,Муж,"Начальное, среднее образование",24800.8800,356,0.00960,69.665393
806,Столица,20-29,50000-70000,Муж,"Начальное, среднее образование",46031.1165,666,0.01796,69.115791
232,Регионы,20-29,50000-70000,Муж,"Начальное, среднее образование",30705.3405,449,0.01211,68.386059
813,Столица,20-29,70000-90000,Муж,Высшее образование,28378.9755,418,0.01127,67.892286
807,Столица,20-29,50000-70000,Муж,Неполное высшее образование,27758.6285,414,0.01116,67.049827
804,Столица,20-29,50000-70000,Муж,Высшее образование,38509.4000,580,0.01564,66.395517
795,Столица,20-29,30000-50000,Муж,Высшее образование,24750.9360,390,0.01052,63.463938
230,Регионы,20-29,50000-70000,Муж,Высшее образование,24527.3120,393,0.01060,62.410463


In [44]:
a = df[(df.region == "Столица") & (df.age_group == "20-29") & (df.income_group == "50000-70000") & (df.gender_cd == "Муж") & (df.education_level == "Начальное, среднее образование")]
b = df[(df.region == "Регионы") & (df.age_group == "20-29") & (df.income_group == "50000-70000") & (df.gender_cd == "Муж") & (df.education_level == "Начальное, среднее образование")]

b.id.count()

897

In [45]:
from scipy.stats import ttest_ind

# Гипотеза 
ttest_ind(a.good_price, b.good_price)

TtestResult(statistic=-0.13479116615449288, pvalue=0.8927890412866493, df=2251.0)

In [60]:
# Кол-во уникальных пользователей
a = df.drop(df[df.client_id.duplicated()].index)
a = a.groupby([
    "region", "age_group",
    "income_group", "gender_cd",
    "education_level"
]).education_level.value_counts().reset_index()

# Кол-во покупок
b = df.groupby([
    "region", "age_group",
    "income_group", "gender_cd",
    "education_level"
]).education_level.value_counts().reset_index()

a["conver"] = (b["count"] / a["count"])
a[a["count"] > 100].sort_values("conver", ascending=False).head(20)

,region,age_group,income_group,gender_cd,education_level,count,conver
149,Регионы,20-29,110000-130000,Муж,"Начальное, среднее образование",118,3.330508
147,Регионы,20-29,110000-130000,Муж,Высшее образование,140,3.000000
253,Регионы,20-29,<30 000,Муж,Высшее образование,342,2.511696
740,Столица,20-29,170000-190000,Муж,Высшее образование,149,1.953020
255,Регионы,20-29,<30 000,Муж,"Начальное, среднее образование",474,1.892405
256,Регионы,20-29,<30 000,Муж,Неполное высшее образование,343,1.469388
822,Столица,20-29,90000-110000,Муж,Высшее образование,238,1.042017
795,Столица,20-29,30000-50000,Муж,Высшее образование,390,1.038462
934,Столица,30-39,50000-70000,Муж,Высшее образование,358,0.879888
962,Столица,30-39,<30 000,Муж,"Начальное, среднее образование",132,0.856061


In [47]:
r = df[(df.region == "Столица") & (df.age_group == "80-89") & (df.income_group == ">300 000") & (df.gender_cd == "Жен") & (df.education_level == "Высшее образование")]
r

,id,client_id,order_day,category_id,category_name,good_id,good_name,good_price,good_cnt,gender_cd,age,education_level,city_nm,monthly_income_amt,steam_popularity_score,region,age_group,income_group
80969,1500539077,5-33SADLHBM,2023-04-11,30659,Казуальные игры,1454888,"Love, Money, Rock'n'Roll",360.0,1.0,Жен,81,Высшее образование,Москва,317321,2559.0,Столица,80-89,>300 000


In [62]:
a = df[df.education_level == "Высшее образование"]
b = df[df.education_level == "Начальное, среднее образование"]

ttest_ind(a.good_price, b.good_price)

TtestResult(statistic=1.484441317878549, pvalue=0.13769724762542496, df=58724.0)